
You have text data and want to tag each word or character with its part of
speech.

Use NLTK’s pre-trained parts-of-speech tagger:


In [2]:
# Load libraries
from nltk import pos_tag
from nltk import word_tokenize


In [3]:
# Create text
text_data = "Chris loved outdoor running"
# Use pre-trained part of speech tagger
text_tagged = pos_tag(word_tokenize(text_data))
# Show parts of speech
text_tagged


[('Chris', 'NNP'), ('loved', 'VBD'), ('outdoor', 'RP'), ('running', 'VBG')]

The output is a list of tuples with the word and the tag of the part of speech.
NLTK uses the Penn Treebank parts for speech tags. Some examples of the Penn
Treebank tags are

![](./NLTKtag.jpg)